In [ ]:
import pandas as pd
import json
from itertools import chain

files = [
    # "js_2023_02_06/warm/query_responses.json",
    # "js_2023_02_06/cold/query_responses.json",
    # "merlin/cmi-cache-only/all-commands/main/full_responses.json",
    # # "merlin/cmi-cache-only/all-commands/merlin-domains/full_responses.json",
    # # "merlin/cmi-cache-only/complete-prefix/main/full_responses.json",
    # # "merlin/cmi-cache-only/complete-prefix/merlin-domains/full_responses.json",
    # "merlin/full-cache/all-commands/main/full_responses.json",
    # "merlin/full-cache/all-commands/merlin-domains/full_responses.json",
    # "irmin-15-10-2025/10/cmi-cache-only/all-commands/data-race-free-merlin-domains",
    # "irmin-15-10-2025/10/cmi-cache-only/all-commands/main",
    "irmin-16-10-2025/cmi-cache-only/irmin/all-commands/data-race-free-merlin-domains/full_responses.json",
    "irmin-16-10-2025/cmi-cache-only/irmin/all-commands/main/full_responses.json",
    "irmin-16-10-2025/full-cache/irmin/all-commands/data-race-free-merlin-domains/full_responses.json",
    "irmin-16-10-2025/full-cache/irmin/all-commands/main/full_responses.json",
    "irmin-16-10-2025/cmi-cache-only/irmin-pack/all-commands/main/full_responses.json",
    "irmin-16-10-2025/full-cache/irmin-pack/all-commands/data-race-free-merlin-domains/full_responses.json",
    "irmin-16-10-2025/full-cache/irmin-pack/all-commands/main/full_responses.json"
]


def frame_from_file(file):
    data = pd.read_json(file, lines = True)

    resp = list(chain.from_iterable(data['responses']))

    resp_with_id = []

    for _, row in data.iterrows():
        sample_id = row['sample_id']
        unix_time = row['unix_time']
        for resp in row['responses']:
            resp_copy = resp.copy()
            resp_copy['sample_id'] = sample_id
            resp_copy['unix_time'] = unix_time
            resp_with_id.append(resp_copy)

    return pd.json_normalize(resp_with_id, sep='.')


In [7]:
data = pd.read_json("merlin/full-cache/all-commands/merlin-domains/full_responses.json", lines = True)

resp = list(chain.from_iterable(data['responses']))
resp_with_id = []
for _, row in data.iterrows():
    sample_id = row['sample_id']
    for resp in row['responses']:
        resp_copy = resp.copy()
        resp_copy['sample_id'] = sample_id
        resp_with_id.append(resp_copy)
pd.json_normalize(resp_with_id, sep='.')

,class,value,notifications,heap_mbytes,query_num,sample_id,timing.clock,timing.cpu,timing.query,timing.pp,...,cache.cmt.hit,cache.cmt.miss,cache.cmi.hit,cache.cmi.miss,value.file,value.pos.line,value.pos.col,value.entries,value.context,cache.typer
0,return,[],[],4480,0,4238,16,16,2,0.0,...,0.0,0.0,27.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,return,NaN,[],4479,0,4237,17,17,3,0.0,...,0.0,0.0,27.0,0.0,std.ml,361.0,11.0,NaN,NaN,NaN
2,return,NaN,[],4477,0,4236,17,17,3,0.0,...,2.0,0.0,27.0,0.0,stdlib.ml,83.0,9.0,NaN,NaN,NaN
3,return,NaN,[],4475,0,4235,19,19,3,0.0,...,0.0,0.0,27.0,0.0,NaN,NaN,NaN,"[{'name': 's', 'kind': 'Value', 'desc': '', 'i...",None,NaN
4,return,NaN,[],4473,0,4234,18,18,3,0.0,...,0.0,0.0,27.0,0.0,NaN,NaN,NaN,"[{'name': '()', 'kind': 'Constructor', 'desc':...",None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,return,"[{'start': {'line': 106, 'col': 6}, 'end': {'l...",[],60,0,4,10,12,1,0.0,...,84.0,0.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3995,return,"[{'start': {'line': 85, 'col': 45}, 'end': {'l...",[],58,0,3,10,13,11,0.0,...,62.0,0.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3996,return,"[{'start': {'line': 76, 'col': 53}, 'end': {'l...",[],56,0,2,11,14,13,0.0,...,139.0,29.0,68.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN
3997,error,Destruct not allowed on arrow type,[],54,0,1,29,40,40,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def print_percentiles(frame, file):
    percentiles = [0.25, 0.5, 0.75, 0.90, 0.95]


    desc_clock = frame['timing.clock'].describe(percentiles=percentiles)
    desc_cpu = frame['timing.cpu'].describe(percentiles=percentiles)
    desc_reader = frame['timing.reader'].describe(percentiles=percentiles)
    desc_ppx = frame['timing.ppx'].describe(percentiles=percentiles)
    desc_typer = frame['timing.typer'].describe(percentiles=percentiles)
    desc_query = frame['timing.query'].describe(percentiles=percentiles)
    desc_unix = frame['unix_time'].describe(percentiles=percentiles)


    df_clock  = pd.DataFrame(desc_clock).T
    df_cpu    = pd.DataFrame(desc_cpu).T
    df_reader = pd.DataFrame(desc_reader).T
    df_ppx    = pd.DataFrame(desc_ppx).T
    df_typer  = pd.DataFrame(desc_typer).T
    df_query  = pd.DataFrame(desc_query).T
    df_unix   = pd.DataFrame(desc_unix).T


    df_clock.index  = ["timing.clock"]
    df_cpu.index    = ["timing.cpu"]
    df_reader.index = ["timing.reader"]
    df_ppx.index    = ["timing.ppx"]
    df_typer.index  = ["timing.typer"]
    df_query.index  = ["timing.query"]
    df_unix.index   = ["unix_time"]

    # parts = file.split("/")
    # cache_part = parts[1]
    # cmd_part = parts[2]
    # branch_part = parts[3]
    # print(f"Branch: {branch_part}")
    # print(f"Cache: {cache_part}")
    # print(f"Commands: {cmd_part}")

    print(file)


    display(pd.concat([ df_unix, df_clock, df_cpu, df_reader, df_ppx, df_typer, df_query]))
    print("\n\n")

for f in files:
    frame = frame_from_file(f)
    print_percentiles(frame, f)

irmin-16-10-2025/cmi-cache-only/irmin/all-commands/data-race-free-merlin-domains/full_responses.json


,count,mean,std,min,25%,50%,75%,90%,95%,max
unix_time,4136.0,71.267170,68.417255,3.941059,55.685043,60.801983,70.221961,92.695355,137.940466,3225.539923
timing.clock,4136.0,67.971470,68.569414,2.000000,53.000000,57.000000,67.000000,90.000000,135.250000,3223.000000
timing.cpu,4136.0,68.769342,84.987672,2.000000,52.000000,56.000000,65.000000,93.000000,142.000000,4428.000000
timing.reader,3637.0,4.190542,8.963914,0.000000,1.000000,1.000000,4.000000,10.000000,17.000000,84.000000
timing.ppx,3637.0,43.949684,12.783418,0.000000,41.000000,48.000000,50.000000,52.000000,54.000000,101.000000
timing.typer,3637.0,4.804509,28.651034,0.000000,0.000000,0.000000,0.000000,5.000000,17.000000,551.000000
timing.query,4136.0,22.114362,78.051248,0.000000,4.000000,7.000000,19.000000,57.000000,83.000000,4401.000000





irmin-16-10-2025/cmi-cache-only/irmin/all-commands/main/full_responses.json


,count,mean,std,min,25%,50%,75%,90%,95%,max
unix_time,4136.0,76.276617,77.626812,4.661083,55.377185,60.277939,69.582999,119.676471,173.190534,3168.94412
timing.clock,4136.0,73.116054,77.766065,3.000000,52.000000,57.000000,66.000000,117.000000,170.500000,3166.00000
timing.cpu,4136.0,71.725822,77.392873,2.000000,52.000000,56.000000,64.000000,115.000000,167.500000,3165.00000
timing.reader,4136.0,3.494923,7.372026,0.000000,1.000000,1.000000,3.000000,7.000000,14.000000,75.00000
timing.ppx,4136.0,44.067698,12.021414,0.000000,41.000000,47.000000,49.000000,51.000000,53.000000,130.00000
timing.typer,4136.0,20.283849,53.296299,0.000000,2.000000,4.000000,12.000000,55.000000,97.000000,503.00000
timing.query,4136.0,3.843569,46.956513,1.000000,2.000000,3.000000,3.000000,4.000000,5.000000,3013.00000





irmin-16-10-2025/full-cache/irmin/all-commands/data-race-free-merlin-domains/full_responses.json


,count,mean,std,min,25%,50%,75%,90%,95%,max
unix_time,4136.0,58.698574,57.193022,4.530907,45.954466,58.448553,62.309802,71.713924,85.889876,3350.406885
timing.clock,4136.0,55.091393,57.286369,2.000000,43.000000,55.000000,58.000000,68.000000,83.000000,3348.000000
timing.cpu,4136.0,55.187137,68.037164,2.000000,43.000000,54.000000,58.000000,69.000000,87.000000,4060.000000
timing.reader,3637.0,4.744295,10.539154,0.000000,1.000000,1.000000,4.000000,12.000000,20.000000,87.000000
timing.ppx,3637.0,43.230410,13.672732,0.000000,34.000000,49.000000,51.000000,52.000000,55.000000,134.000000
timing.typer,3637.0,0.257630,4.627179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,158.000000
timing.query,4136.0,12.543520,67.049990,1.000000,3.000000,3.000000,5.000000,46.000000,56.000000,4034.000000





irmin-16-10-2025/full-cache/irmin/all-commands/main/full_responses.json


,count,mean,std,min,25%,50%,75%,90%,95%,max
unix_time,4136.0,63.240375,58.285553,3.554821,53.717613,58.921456,67.077696,75.470448,93.907475,3244.596958
timing.clock,4136.0,59.912959,58.290800,2.000000,51.000000,55.000000,64.000000,72.000000,91.250000,3240.000000
timing.cpu,4136.0,56.491538,57.742542,1.000000,50.000000,53.000000,57.000000,67.000000,86.250000,3239.000000
timing.reader,4136.0,3.752418,8.206737,0.000000,1.000000,1.000000,3.000000,8.000000,15.000000,74.000000
timing.ppx,4136.0,44.861944,12.088210,0.000000,43.000000,48.000000,50.000000,52.000000,54.000000,107.000000
timing.typer,4136.0,3.589458,21.560920,0.000000,0.000000,0.000000,1.000000,4.000000,11.000000,493.000000
timing.query,4136.0,4.147002,49.666463,0.000000,2.000000,3.000000,4.000000,5.000000,6.000000,3185.000000


In [3]:
sorted = comp_times.sort_values(ascending=False)

sorted.head(20)

NameError: name 'comp_times' is not defined